<a href="https://colab.research.google.com/github/Adritomustafa153/Textclasification_using_BERT/blob/main/FYDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [ ]:
pip install torch

In [ ]:
pip install -U tensorflow-text

In [4]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
pip install --upgrade tensorflow


In [5]:
df = pd.read_csv('dataset.csv')

df.head()

,Name,Behaviour
0,Engine Problems,What leads to the engine emitting a burning od...
1,Engine Problems,Why does my car suddenly halt and leave me str...
2,Engine Problems,What is responsible for the underpowered accel...
3,Air Conditioning and Heating Troubles,What might be limiting the heater's proper fun...
4,Air Conditioning and Heating Troubles,Why isn't my car's heater operating at its ful...


In [6]:
df.shape

(1299, 2)

In [7]:
from sklearn.model_selection import train_test_split

x = df['Name']
y = df['Behaviour']



In [8]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [9]:
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [22]:
def get_sentence_embeding(sentence):
  pre_processe_text = bert_preprocess(sentence)
  return bert_encoder(pre_processe_text)['pooled_output']

In [ ]:
get_sentence_embeding([
    "Hello this is my fydp work"
])

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype = tf.string, name ='text')
pre_processe_text = bert_preprocess(text_input)
outputs = bert_encoder(pre_processe_text)

l = tf.keras.layers.Dropout(0.3,name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation= "sigmoid", name ="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs=[l])
model.summary()

In [21]:
model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.fit(X_train,y_train,epochs=5)